In [ ]:
import time
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, plot_confusion_matrix
import plotly.graph_objects as go
df = pd.read_csv('./dataset/Apply_Rate_2019.csv').drop(['class_id'], axis=1)
df.drop_duplicates(inplace=True)

In [ ]:
fig = go.Figure([go.Bar(x=df.columns, y=df.isnull().sum())])
fig.update_layout(xaxis_type='category', title_text='Number of null data', title_x=0.5)
fig.show()  # imbalanced

In [ ]:
x = df.describe()

# this thing cannot show correctly in csvode, I've saved it to .images/
fig = go.Figure()
fig.add_trace(go.Box())
IQR = x.loc['75%',]-x.loc['25%']
fig.update_traces(q1=x.loc['25%',], median=x.loc['50%',], q3=x.loc['75%',], mean=x.loc['mean',], lowerfence=x.loc['min',],
                  upperfence=x.loc['75%',]+IQR, sd=x.loc['std',])
fig.update_layout(title_text='data distribution within fences', title_x=0.5)
fig.show()

In [ ]:
fig = go.Figure([go.Bar(x=df['apply'].unique(), y=df['apply'].value_counts())])
fig.update_layout(xaxis_type='category', title_text='Apply frequency', title_x=0.5)
fig.show()  # imbalanced

In [ ]:
# start fking preprocessing
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# fill missing values by MICE, then change city_match back to 0/1
# because directly assign 0 to city_match will worsen the result.
df.iloc[:,:3] = IterativeImputer().fit_transform(df.iloc[:,:3])  # fillna probabilities
df.iloc[:,5:6] = IterativeImputer().fit_transform(df.iloc[:,5:6])  # fillna city_match, not relevant to others 
df['city_match'] = df['city_match'].round()

df.describe()

In [ ]:
print('number of NaN in title_proximity_tfidf:',len(df[df['title_proximity_tfidf'].isnull()]))
print('number of NaN in description_proximity_tfidf:',len(df[df['description_proximity_tfidf'].isnull()]))
print('number of NaN in city_match:',len(df[df['city_match'].isnull()]))

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
#from imblearn.over_sampling import SMOTE

# train_x, test_x = train_test_split(df, test_size=0.2, random_state=4211)
test_x = df[df['search_date_pacific']=='2018-01-27'].drop(['search_date_pacific'], axis=1)
train_x = df[df['search_date_pacific']!='2018-01-27'].drop(['search_date_pacific'], axis=1)
train_y = train_x.pop('apply')
test_y = test_x.pop('apply')

# Normalisation using the sklearn StandardScaler. Set the mean to 0 and sd to 1.
transform_columns = ['title_proximity_tfidf', 'description_proximity_tfidf', 'main_query_tfidf', 'query_jl_score', 'query_title_score', 'job_age_days']
ct = ColumnTransformer(
        remainder='passthrough',
        transformers=[('std', StandardScaler(), transform_columns)])
train_x = ct.fit_transform(train_x)
test_x = ct.fit_transform(test_x)

# after ColumnTransformm, city_match is at the ENDDDDDDD of the dataframe, this took me hours to figure out :)
x_columns = ['title_proximity_tfidf', 'description_proximity_tfidf', 'main_query_tfidf', 'query_jl_score', 'query_title_score', 'job_age_days', 'city_match']
train_x = pd.DataFrame(train_x, columns=x_columns)
test_x = pd.DataFrame(test_x, columns=x_columns)

# undersampling to reduce imbalance
#train_x, train_y = SMOTE(sampling_strategy=0.2, random_state=0).fit_resample(train_x, train_y)  # not much different
train_x, train_y = RandomUnderSampler(random_state=0).fit_resample(train_x, train_y)

train_x.describe()

In [ ]:
fig = go.Figure([go.Bar(x=train_y.unique(), y=train_y.value_counts())])
fig.update_layout(xaxis_type='category', title_text='Apply frequency in training set', title_x=0.5)
fig.show()  # imbalanced

In [ ]:
def report(clf):
    pred_y = clf.predict(test_x)

    print(classification_report(test_y, pred_y))
    disp = plot_confusion_matrix(clf, test_x, test_y)
    disp.figure_.suptitle("Confusion Matrix")

def train(clf):
    start = time.time()
    clf.fit(train_x, train_y)
    print(time.time()-start)
    report(clf)

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(80, 60, ), learning_rate='adaptive', early_stopping=True, random_state=4211)
train(mlp)

In [ ]:
from sklearn.model_selection import GridSearchCV  # this thing cpu intensive.
tuned_parameters = {
    'hidden_layer_sizes': [(120,), (80, 60,), (60, 40, 20,)]
    'solver': ['lbfgs', 'sgd', 'adam'],
    'learning_rate': ['invscaling', 'adaptive'],
    'learning_rate_init': [0.001, 0.002, 0.005],
    'momentum', [0.9, 0.8]
    'early_stopping': [True],
    'random_state': [4211]
}

clf = GridSearchCV(MLPClassifier(), tuned_parameters, n_jobs=-1, verbose=2)
train(clf)

In [ ]:
clf = LinearSVC(random_state=4211)
train(clf)

In [ ]:
clf = LinearSVC(multi_class='crammer_singer', random_state=4211)
train(clf)

In [ ]:
from sklearn.model_selection import GridSearchCV  # this thing cpu intensive.
tuned_parameters = {'C': [0.01, 0.05, 0.1, 0.5, 1, 5],
                    'multi_class': ['crammer_singer'],
                    'random_state': [4211]
                    }

clf = GridSearchCV(LinearSVC(), tuned_parameters, n_jobs=-1, verbose=2)
train(clf)